In [ ]:
# This excercise explains core components of the langchain, will demo the following concepts:

# - LLM Models and model wrappers (here we use open AI LLM)
# - Prompt templates (which are used to generate prompts for the models)
# - Chains (which get input from one chain to another and helps in chaining multipls repsponses and represent it in a more meaningful response)
# - Embeddings and vector stores
# - Agents

# reference: https://www.youtube.com/watch?v=aywZrzNaKjs

True

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # This is to load your .env file with your credentials

1. LLM Models:

In [ ]:
# this is an example of how to use the OpenAI API with the langchain library
# you can install langchain with pip install langchain
# and you can install OpenAI with pip install openai
from langchain import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo")
llm("explain large language models in one sentence")

In [23]:
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain.chat_models import ChatOpenAI

In [24]:
# this is an example of how to use the ChatOpenAI class with the langchain library
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data"),
]
response = chat(messages)
print(response.content, end='\n')

Sure, here is an example Python script that trains a simple neural network on simulated data using the popular library TensorFlow:

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 2)
y = np.random.randint(0, 2, 1000)

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')
```

In this script, we first generate simulated data with 2 features and binary labels. We then define a simple neural network with one hidden layer and an output layer. We compile the model with binary cross-entropy loss and train it on

2. Prompts:

In [25]:
# this is an example of how to use the PromptTemplate class with the langchain library

from langchain import PromptTemplate

template = """"
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in couple of lines.
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template)

In [26]:
prompt

PromptTemplate(input_variables=['concept'], output_parser=None, partial_variables={}, template='"\nYou are an expert data scientist with an expertise in building deep learning models.\nExplain the concept of {concept} in couple of lines.\n', template_format='f-string', validate_template=True)

In [27]:
# call llm with the prompt
llm(prompt.format(concept="gradient descent"))

'Gradient descent is an optimization algorithm used to minimize the error of a model by adjusting its parameters iteratively. It works by calculating the gradient of the loss function with respect to each parameter and moving in the direction opposite to the gradient to reach the optimal set of parameters that minimize the error.'

3. Chains:

In [28]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt) # LLMChain is a class that chains multiple LLMs together

# Run the chain only specifying the input variables
print(chain.run("autoencoder"))

An autoencoder is a type of neural network that learns to encode input data into a lower-dimensional representation and reconstruct the original input from this encoded representation. It is commonly used for dimensionality reduction, data compression, and anomaly detection tasks in machine learning.


In [30]:
second_prompt = PromptTemplate(
    input_variables=["machine_learning_concept"],
    template="Turn the concept description of {machine_learning_concept} and explain it to me in layman's terms.")
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
from langchain.chains import SimpleSequentialChain
total_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the total chain with the input "autoencoder" and print the resulting explanation.
explanation = total_chain.run("autoencoder")

print(explanation)



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of neural network that learns to encode input data into a lower dimensional representation, known as the "code" or "latent space", before decoding it back into the original input data. This unsupervised learning technique is often used for tasks such as dimensionality reduction, anomaly detection, and data denoising.
An autoencoder is like a translator that takes a complex piece of information, like a paragraph, and simplifies it into a shorter version, known as a code. It then translates this code back into the original paragraph. This can be helpful for organizing data, finding errors, or cleaning up messy information.

> Finished chain.
An autoencoder is like a translator that takes a complex piece of information, like a paragraph, and simplifies it into a shorter version, known as a code. It then translates this code back into the original paragraph. This can be helpful for organizing data, finding errors, or 

4. Embeddings and Vector stores:

In [32]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap= 0)

texts = text_splitter.create_documents([explanation])

In [33]:
texts

[Document(page_content='An autoencoder is like a translator that takes a complex piece of information, like a paragraph, and', metadata={}),
 Document(page_content='simplifies it into a shorter version, known as a code. It then translates this code back into the', metadata={}),
 Document(page_content='original paragraph. This can be helpful for organizing data, finding errors, or cleaning up messy', metadata={}),
 Document(page_content='information.', metadata={})]

In [37]:
texts[0].page_content

'An autoencoder is like a translator that takes a complex piece of information, like a paragraph, and'

In [ ]:
# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model_name="text-embedding-ada-002")

In [68]:
import tiktoken

embeddings = tiktoken.get_encoding("cl100k_base")
query_results = embeddings.encode(texts[0].page_content)
# query_results = embeddings.embed_query(texts[0].page_content)
query_results

[2127,
 3313,
 28106,
 374,
 1093,
 264,
 46588,
 430,
 5097,
 264,
 6485,
 6710,
 315,
 2038,
 11,
 1093,
 264,
 14646,
 11,
 323]

In [78]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY")
)

In [ ]:
index_name = "langchain-demo"

# Check if the index already exists, if not, create it
if index_name not in pinecone.list_indexes():
	pinecone.create_index(index_name, dimension=len(embeddings.encode(texts[0].page_content)))

search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [ ]:
query = "explain autoencoder"
results = search.similarity_search(query)

5. Agents:

In [82]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100),
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.run("print('Hello, World!')")

C:\Users\rrajeshrajappan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\llms\openai.py:170: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\rrajeshrajappan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\llms\openai.py:624: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new AgentExecutor chain...
The code provided will print "Hello, World!" to the console.
Action: Python REPL
Action Input: print('Hello, World!')
Observation: Hello, World!

Thought:The code executed successfully and printed the expected output.
Final Answer: Hello, World!

> Finished chain.


'Hello, World!'